## Imports

In [ ]:
import pandas as pd
import numpy as np

# CNN
import keras

# Graph
import seaborn as sns

# Model
from sklearn.model_selection import train_test_split

# AWS
import boto3

Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
